#Prepare environment

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd

#Upload datasets

> Define products to be taken into account for credited conversions

>/!\ Which products do you want to take into account????

In [9]:
#update gamme produits that need to be taken into account for conversions credit
#http://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe
gamme_prod = ['DN', 'DI', 'PD', 'TI', 'TN']

# CPM 
cpm = 6.0

> Name of the files to be uploaded

> These should be hosted in the root folder

In [13]:
#name you file as the one you uploaded in cloud 9 - file_quanti 
adServerFile = 'Report_Quantcast_mobilite_Standard_v1_M88Z5cUHryFJpRP_AA1S.xlsx'

#name your 3PAS file as the one you uploaded in cloud 9 - file_quali 
detailedFile = 'Report_Quantcast_mobilite_conversion_view_PNJjl0461vpIzpMgHrA7.csv'

#Defining the function that'll be used to generate the final CSV report

In [11]:
def LP(gamme_prod, adServerFile, detailedFile, QC_cpm):    
    
    #first file to be uploaded details of products part of each conversion
    detailedConversions = pd.read_csv(detailedFile, dtype=object)

    #We need to update the date format and strip hours
    #http://stackoverflow.com/questions/26387986/strip-time-from-an-object-date-in-pandas
    detailedConversions['conversion_date'] = pd.to_datetime(detailedConversions['Conversion date-hour'])
    detailedConversions['conversion_date'] = detailedConversions['conversion_date'].apply(lambda x:x.date().strftime('%Y-%m-%d'))
    #then upload the second file. this is the regular 3PAS files with every conversions to which we contributed. 
    xls_file = pd.ExcelFile(adServerFile)
    adServerTable = xls_file.parse('DataView')

    #create functio that will be used to create matching key by concatenating several columns
    #http://stackoverflow.com/questions/23444858/concatenate-column-values-in-pandas-dataframe-with-nan-values
    def concat(*args):
        strs = [str(arg) for arg in args if not pd.isnull(arg)]
        return '_'.join(strs) if strs else np.nan

    np_concat = np.vectorize(concat)

    #We need to define the key to join both file. 
    #first 3PAS report Key
    adServerTable['key'] = np_concat(adServerTable['Date'], adServerTable['Site/Offer ID'], adServerTable['Insertion ID'], adServerTable['Creative ID'])
    #QA the created key

    detailedConversions['key'] = np_concat(detailedConversions['conversion_date'], detailedConversions['Site-Offer ID'], detailedConversions['Insertion ID'], detailedConversions['Creative ID'])

    # function aimed at cleaning revenue figures

    def cleanrev(col):
        if col == '-':
            return 0.0
        else:
            return float(col)

    # function aimed at cleaning products part of the orders

    def cleanGamme(col,gamme_prod):
        items = list(set(col.split("|")) )
        if len(items) == 1:
            return items[0]
        elif re.search('$|^'.join(gamme_prod),col):
            return "Mixte"
        else:
            return "Mixte_non_comptabilise"

    # function aimed at associating products and unit price
    # /!\ We need to verify every product are added and quantity matches

    def cleanProduitMix(df):
        items = df[0].split("|")
        prices = df[1].split("|")
        quantity = df[2].split("|")
        if len(items) == 1:
            return items[0] + '-%s' %float(prices[0]) #*float(quantity[0]))
        else:
    #        mix = '%s-' %len(items)
            l = len(items)
            mix = ''
            for i in range(0,len(items)-2):
                mix = mix + items[i] + '-%s|' %float(prices[i]) #*float(quantity[i]))
            return mix + items[l-1] + '-%s' %float(prices[l-1]) #*float(quantity[l-1]))

    #define the same function without tacking into account the quantity section    
    def cleanProduitMixWOQuantities(df):
        items = df[0].split("|")
        prices = df[1].split("|")
        quantity = df[2].split("|")
        if len(items) == 1:
            return items[0] + '-%s' %(float(prices[0]))
        else:
            l = len(items)
            mix = ''
            for i in range(0,len(items)-2):
                mix = mix + items[i] + '-%s|' %(float(prices[i]))
            return mix + items[l-1] + '-%s' %(float(prices[l-1]))

    #function aimed at concatenating several columns separated by '-'
    def concatenate(df):
        if len(df)>0:
            c = ''
            l = len(df)-1
            for i in range (0, l-1):
                c = c + df[i] + '-'
            return c + df[l]
        else:
            return ''

    #function to be used when filtering out products not in required
    def filter_products(x):
        if re.search('|'.join(gamme_prod),x):
            return "true"
        else:
            return "false"

    details3PAS = detailedConversions[['key','nom_site','gamme_produit', 'prix_produit','nombre_produit' ,'prix_ht_zanox']]
    details3PAS = pd.DataFrame(details3PAS).dropna()

    details3PAS['revenue_transaction'] = details3PAS.prix_ht_zanox.apply(lambda x: cleanrev(x))
    details3PAS['panier'] = details3PAS.gamme_produit.apply(lambda x: cleanGamme(x, gamme_prod))
    
    dataSet3PAS= details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)
    dataSet3PAS['selected'] = dataSet3PAS['panier'].apply(lambda x: filter_products(x))
    dataSet3PAS = dataSet3PAS[dataSet3PAS['selected'] == 'true']
    
    transactions = dataSet3PAS.groupby(['key', 'nom_site']).agg({
                                         'panier': 'count',
                                         'revenue_transaction': sum
                                                   }).reset_index()    
    
    transactions.columns = ['key', 'nom_site', 'relevant_conversions', 'total_revenue']

    details3PAS['order'] = details3PAS[['gamme_produit','prix_produit','nombre_produit']].apply(cleanProduitMixWOQuantities, axis = 1)
    dataSet3PAS= details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)
    dataSet3PAS = dataSet3PAS[dataSet3PAS.panier.isin(gamme_prod)]

    #create a column with only porduct and unit prices
    s = details3PAS['order'].str.split("|", expand=True).stack() #.reset_index()
    i = s.index.get_level_values(0)
    df = details3PAS.loc[i].copy()
    df["order"] = s.values
    #df.head(20)
    
    #make two columns out of this. might be tricky if we want to keep the original DF 
    #df_bis = pd.DataFrame(df['order'].str.split("-", 1).tolist(), columns = ['product', 'price'], index = df.index) #.stack() #.reset_index()
    
    df_bis = df 
    #if cleanProduitMix function used
    #df_bis["product"], df_bis["price"], df_bis["quantity"] = zip(*df_bis["order"].str.split('-',2))
    #if cleanProduitMixWOQuantities function used
    df_bis["product"], df_bis["price"] = zip(*df_bis["order"].str.split('-',1))
    
    
    df_bis["product"] = df_bis["product"].apply(lambda x: str(x))
    df_bis["price"] = df_bis["price"].apply(lambda x: float(x))
    
    #uncomment if quantity is being taken into account
    #df_bis["quantity"] = df_bis["quantity"].apply(lambda x: float(x))

    #compute revenue for each product sold
    #uncomment the relevant line whether you take into account the quantity
    #df_bis['revenue_product'] = df_bis['price'] * df_bis['quantity']
    df_bis['revenue_product'] = df_bis['price'] * 1
    df_bis = df_bis[['key', 'nom_site', 'product', 'revenue_product']]

    #keep only products concerned by our advertisign campaign
    df_final = df_bis
    #using our function defined above
    df_final["product_bis"] = df_final["product"].apply(lambda x: filter_products(x))
    
    #filtering out false products (products not concerned by the campaign)
    df_final = df_final[df_final['product_bis'] == "true"]

    
    df_final_bis = df_final.groupby(['key', 'nom_site']).agg({
                                         'revenue_product':sum
                                                   }).reset_index()   

    df_final_cleaned = pd.merge(df_final_bis,transactions, how= 'inner', on = ['key', 'nom_site'] ,left_index=False,right_index=False)

####################################################################################################################################
####################################################################################################################################
    #transactions = dataSet3PAS.groupby(['key']).count()
    #revenue = dataSet3PAS.groupby(['key']).sum()
    #ds = pd.DataFrame(transactions,revenue)
    #transactions = pd.DataFrame(dataSet3PAS.groupby(["key"]).sum()["revenue_transaction"])
    #revenue = pd.DataFrame(dataSet3PAS.groupby(["key"]).count()["panier"])
    #ds = transactions.join(revenue).reset_index()
    #c = np.append('key', 'revenue', 'conversions',axis =0)
    #ds.columns = ['key', 'revenue', 'conversions']

####################################################################################################################################
####################################################################################################################################

    final_ds = pd.merge(adServerTable,df_final_cleaned, on='key')

    final_ds = final_ds[['Date', 'Site/Offer ID','Site/Offer', 'Insertion ID', 'Insertion', 'Creative ID', 'Creative', 'Creative Sizes','Imp.', 'Clicks','relevant_conversions', 'revenue_product','total_revenue', 'key' ]]

    final_ds['cost'] = QC_cpm * final_ds['Imp.'] / 1000
    
    final_ds['cpa'] = final_ds['cost'] / final_ds['relevant_conversions']
    final_ds['roi'] = final_ds['revenue_product'] / final_ds['cost']
    
    final_ds.to_csv('3pas_report_final.csv', index=False,encoding='utf-8')
    print('Done')


#finally run the function so that report is generated in the root folder

In [12]:
LP(gamme_prod, adServerFile, detailedFile, cpm)

Done
